<a href="https://colab.research.google.com/github/MelihKlc/Deep-Learning/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP with TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences or text)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-08-15 13:06:04--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-08-15 13:06:04 (58.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback , plot_loss_curves , compare_historys , unzip_data , walk_through_dir , calculate_results

# Get the text data

The dataset we are going to use it NLP dataset for text samples tweets labelled as disaster or not disaster

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-08-15 13:21:49--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.253.63.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-08-15 13:21:50 (136 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

# Become one with the data

In [13]:
# Visualizing our text dataset

import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [15]:
# If target label has 1 , it means that our sentence is about disasters

train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [16]:
# Shuffle training data because of we dont want to be overfitting

train_df_shuffled = train_df.sample(frac = 1 , random_state = 42) # Frac = 1 dememizin sebebi bütün datamızı shuffle etmek istiyoruz.


In [17]:
# What does the our test_dataset look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [20]:
# How many examples of each class?

train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [22]:
# How many total samples?

len(train_df) , len(test_df)

(7613, 3263)

In [41]:
# Lets visualize some random training examples.
import random
random_index = random.randint( 0 , len(train_df) - 5)  # Create random indexes not higher than the total number of samples

for row in train_df[["text" , "target"]][random_index : random_index + 5].itertuples():  # Bu satırda, train_df veri çerçevesinde rastgele seçilen 5 satırı (veya kalan kadar satır varsa) içeren bir alt veri çerçevesini seçiyoruz. Sadece "text" ve "target" sütunlarını seçiyoruz. itertuples() işlevi, seçilen satırları birer birer dönmek için kullanılır.
  _ , text , target = row
  print(f"Target : {target} " , "(real_disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n {text}")
  print("---\n")


Target : 0  (not real disaster)
Text
 I crashed my car into a parked car the other day... #modestmouseremix #truestory
---

Target : 1  (real_disaster)
Text
 f496D mhtw4fnet

Pakistan says army helicopter has crashed in country's restive northwest  - Fox News
---

Target : 1  (real_disaster)
Text
 TTW Today's News: Bin Laden family plane crashed after 'avoiding microlight and landing too far down runway'  http://t.co/BUMzvmwAM3
---

Target : 1  (real_disaster)
Text
 Heard #SKH on the radio for the first time. Almost crashed the car. @5SOS @Ashton5SOS @Luke5SOS @Michael5SOS @Calum5SOS
---

Target : 1  (real_disaster)
Text
 MH370: Intact part lifts odds plane glided not crashed into sea http://t.co/8pdnHH6tzH
---



# Split data into training and validation sets

In [50]:
from sklearn.model_selection import train_test_split

train_sentences , val_sentences , train_labels , val_labels = train_test_split(train_df_shuffled["text"].to_numpy() ,train_df_shuffled["target"].to_numpy() , test_size=0.1 , random_state = 42)

In [52]:
len(train_sentences) , len(val_sentences) , len(train_labels) , len(val_labels)

(6851, 762, 6851, 762)

In [53]:
 # Check the first 10 samples

 train_sentences[ : 10] , train_labels[ : 10]  # Train sentence içindeki cümleler train_labels taki outputlarla eşleşiyor

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

When dealing with a text problem , one of the first things you will have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this namely:
* Tokenization - Direct mapping of token (a token could be a word or a character) to number.
* Embedding - Create a matrix of feature vector for each token (the size of the feature vector can be and this embedding can be learn)

# Text vectorization (tokenization)

In [57]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#ngram substringleri birleştiriyor. Mesela elimizde 'Imagine getting flattened by Kurt Zouma' cümlesi var. ngram = 2 dersek , "Image getting" , "flattened by" , "Kurt Zouma"  gibi ikişer ikişer substringleri birleştiriyor.

# Default values of TextVectorization
text_vectorizer = TextVectorization(max_tokens = 10000 , # How many words in the vocabulary ( automatically add <00V>)
                                                        standardize = "lower_and_strip_punctuation" ,  # Bütün harfleri küçük yapıp noktalama işaretlerini siliyor çünkü bunlar disaster olup olmadıgını anlamamıza yardım eden şeyler değil.
                                                        split = "whitespace"  , # Tokenlar arasında boşluk bırakmak için.
                                                        ngrams = None , # Create a groups of n-word
                                                        output_mode = "int" , # How to map tokens to number
                                                        output_sequence_length = None , # How long do you want your sequences to be. (None atadıgımızda en uzun tokeni alıp bütün tokenlere o lengthi veriyor. Kısa olan sentences ların sonu 0 larla dolduruluyor.)
                                                        pad_to_max_tokens = True
                                    )

In [59]:
train_sentences[0].split()  # Split bu işe yarıyor.

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [58]:
# Find the average number of tokens (words) in the training tweets
# We could just pad them to the longest , but to keep our data small , we will just find the average lenght and then make sure every tweet gets turned into numbers but those list of numbers is the same size as the average throughout every sample.

round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [61]:
# Setup text vectorization variables.

max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # Max length our sequences will be (e.g. How many words from a Tweet does mode see?). Mesela tweet 30 kelime uzunlugunda ama biz bunu 15 atadıgımız icin modelimiz ilk 15 kelimesini görecek

# Uptading our textvectorizer

text_vectorizer = TextVectorization(max_tokens = max_vocab_length ,  # Bizim bütün tweetlerimizden 10000 tane alacak bu 10000 most common lardan oluşacak
                                    output_mode = "int" ,
                                    output_sequence_length = max_length
                                    )


In [62]:
# Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)  # CNN deki fit işlemi gibi artık bütün train sentenceslarımıza text_vectorizerde ne yapılmasını istediysek uygulanacak.

In [ ]:
# We are trying to do here is map the word data that we have to numerical form

In [64]:
# Create a sample sentence and tokenize it

sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])   # Text vectorizer ı biz liste şeklinde oluşturdugumuz icin girdigimiz örnekleri [ ] içinde yazıyoruz.
# Burda shape in 0. indexi sequence i veriyor. 1 tam cümle girdiğimiz için sequence 1.
# Shape in 1. indexi de length veriyor. Max olarak 15 dediğimiz için buna adapt oluyo bizim sample ımız.

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [68]:
# Choose a random sentence from our train_sentences data

import random
random_sentences = random.choice(train_sentences)

print(f"Original sentence : \n {random_sentences} \n\n  Vectorized form of our random sentence: \n")
text_vectorizer([sample_sentence])

Original sentence : 
 The Dress Memes Have Officially Exploded On The Internet http://t.co/yG32yb2jDY 

  Vectorized form of our random sentence: 



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [69]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()  # Get all of the unique words in our training data
top_5_words = words_in_vocab[ : 5 ] # Get the most common words in our data
bottom_5_words = words_in_vocab[ -5 :  ] # Get the least common words

In [71]:
bottom_5_words


['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']